Following [here](https://docs.pydantic.dev/usage/validation_decorator/)

# Validation decorator

ℹ️
> The `validate_arguments` decorator allows the **arguments passed to a function** to be parsed and validated using the
> function's annotations before the function is called. While under the hood this uses the same approach of model 
> creation and initialisation; it provides an extremely easy way to apply validation to your code with minimal boilerplate.

⚠️
> The `validate_arguments` decorator is **in beta**, it has been added to pydantic in v1.5 on a provisional basis.
> It may change significantly in future releases and its interface will not be concrete until v2.

Example:

In [1]:
from pydantic import validate_arguments, ValidationError


@validate_arguments  # NOTE.
def repeat(s: str, count: int, *, separator: bytes = b"") -> bytes:
    b = s.encode()
    return separator.join(b for _ in range(count))


a = repeat("hello", 3)
print(a)
# > b'hellohellohello'

b = repeat("x", "4", separator=" ")  # pyright: ignore
print(b)
# > b'x x x x'

try:
    c = repeat("hello", "wrong")
except ValidationError as exc:
    print(exc)
    """
    1 validation error for Repeat
    count
      value is not a valid integer (type=type_error.integer)
    """

b'hellohellohello'
b'x x x x'
1 validation error for Repeat
count
  value is not a valid integer (type=type_error.integer)


### Argument Types

* Argument types are inferred from type annotations on the function, arguments without a type decorator are considered as `Any`.
* Since `validate_arguments` internally uses a standard `BaseModel`, all types listed in [types](https://docs.pydantic.dev/usage/types/) can be validated, including pydantic models and [custom types](https://docs.pydantic.dev/usage/types/#custom-data-types).
* As with the rest of pydantic, types can be coerced by the decorator before they're passed to the actual function:

In [6]:
import os
from pathlib import Path
from typing import Pattern, Optional

from pydantic import validate_arguments, DirectoryPath


@validate_arguments
def find_file(path: DirectoryPath, regex: Pattern, max=None) -> Optional[Path]:
    for i, f in enumerate(path.glob("**/*")):
        if max and i > max:
            return
        if f.is_file() and regex.fullmatch(str(f.relative_to(path))):
            return f


# note: this_dir is a string here
this_dir = os.path.dirname(".")

print(find_file(this_dir, "^8_val.*"))  # pyright: ignore
# > /Users/samuel/code/pydantic/docs/examples/validation_decorator_main.py
print(find_file(this_dir, "^foobar.*", max=3))  # pyright: ignore
# > None

8_validation_decorator.ipynb
None


A few notes on the above:
* though they're passed as strings, `path` and `regex` are converted to a `Path` object and `regex` respectively by the decorator
* `max` has no type annotation, so will be considered as `Any` by the decorator

> Type coercion like this can be extremely helpful but also confusing or not desired, see below for a discussion of
> `validate_arguments`'s limitations in this regard.

### Function Signatures

The decorator is designed to work with functions using all possible parameter configurations and all possible combinations of these:
* positional or keyword arguments with or without defaults
* variable positional arguments defined via `*` (often `*args`)
* variable keyword arguments defined via `**` (often `**kwargs`)
* keyword only arguments - arguments after `*`,
* positional only arguments - arguments before `, /` (new in Python 3.8)

To demonstrate all the above parameter types:

In [7]:
from pydantic import validate_arguments


@validate_arguments
def pos_or_kw(a: int, b: int = 2) -> str:
    return f"a={a} b={b}"


print(pos_or_kw(1))
# > a=1 b=2
print(pos_or_kw(a=1))
# > a=1 b=2
print(pos_or_kw(1, 3))
# > a=1 b=3
print(pos_or_kw(a=1, b=3))
# > a=1 b=3


@validate_arguments
def kw_only(*, a: int, b: int = 2) -> str:
    return f"a={a} b={b}"


print(kw_only(a=1))
# > a=1 b=2
print(kw_only(a=1, b=3))
# > a=1 b=3


@validate_arguments
def pos_only(a: int, b: int = 2, /) -> str:  # python 3.8 only
    return f"a={a} b={b}"


print(pos_only(1))
# > a=1 b=2
print(pos_only(1, 2))
# > a=1 b=2


@validate_arguments
def var_args(*args: int) -> str:
    return str(args)


print(var_args(1))
# > (1,)
print(var_args(1, 2))
# > (1, 2)
print(var_args(1, 2, 3))
# > (1, 2, 3)


@validate_arguments
def var_kwargs(**kwargs: int) -> str:
    return str(kwargs)


print(var_kwargs(a=1))
# > {'a': 1}
print(var_kwargs(a=1, b=2))
# > {'a': 1, 'b': 2}


@validate_arguments
def armageddon(
    a: int,
    /,  # python 3.8 only
    b: int,
    c: int = None,  # pyright: ignore
    *d: int,
    e: int,
    f: int = None,  # pyright: ignore
    **g: int,
) -> str:
    return f"a={a} b={b} c={c} d={d} e={e} f={f} g={g}"


print(armageddon(1, 2, e=3))
# > a=1 b=2 c=None d=() e=3 f=None g={}
print(armageddon(1, 2, 3, 4, 5, 6, e=8, f=9, g=10, spam=11))
# > a=1 b=2 c=3 d=(4, 5, 6) e=8 f=9 g={'g': 10, 'spam': 11}

a=1 b=2
a=1 b=2
a=1 b=3
a=1 b=3
a=1 b=2
a=1 b=3
a=1 b=2
a=1 b=2
(1,)
(1, 2)
(1, 2, 3)
{'a': 1}
{'a': 1, 'b': 2}
a=1 b=2 c=None d=() e=3 f=None g={}
a=1 b=2 c=3 d=(4, 5, 6) e=8 f=9 g={'g': 10, 'spam': 11}


⚠️ There is no support for the [`typing.overload`](https://docs.python.org/3/library/typing.html#typing.overload) for functions as yet.
See: https://github.com/pydantic/pydantic/issues/1597

### Using Field to describe function arguments

**⚠️ Note: weird pattern, take care!**

* [`Field`](https://docs.pydantic.dev/usage/schema/#field-customisation) can also be used with `validate_arguments`
to provide extra information about the field and validations.
* In general it should be used in a type hint with [`Annotated`](https://docs.pydantic.dev/usage/schema/#typingannotated-fields),
unless `default_factory` is specified, in which case it should be used as the default value of the field:

In [8]:
from datetime import datetime
from pydantic import validate_arguments, Field, ValidationError
from pydantic.typing import Annotated  # pyright: ignore


@validate_arguments
def how_many(num: Annotated[int, Field(gt=10)]):  # NOTE here.
    return num


try:
    how_many(1)
except ValidationError as e:
    print(e)
    """
    1 validation error for HowMany
    num
      ensure this value is greater than 10 (type=value_error.number.not_gt;
    limit_value=10)
    """


@validate_arguments
def when(dt: datetime = Field(default_factory=datetime.now)):  # NOTE here.
    return dt


print(type(when()))
#> <class 'datetime.datetime'>


1 validation error for HowMany
num
  ensure this value is greater than 10 (type=value_error.number.not_gt; limit_value=10)
<class 'datetime.datetime'>


* The alias can be used with the decorator as normal.

### Usage with mypy



The `validate_arguments` decorator should work "out of the box" with `mypy` since it's defined to return a function
with the same signature as the function it decorates.

The only limitation is that since we trick `mypy` into thinking the function returned by the decorator is the same
as the function being decorated; **access to the raw function or other attributes will require `type: ignore**`.

### Validate without calling the function

By default, arguments validation is done **by directly calling the decorated function with parameters**. 

But what if you wanted to validate them without actually calling the function?

To do that you can call the `validate` method bound to the decorated function.


In [10]:
from pydantic import validate_arguments, ValidationError


@validate_arguments
def slow_sum(a: int, b: int) -> int:
    print(f'Called with a={a}, b={b}')
    #> Called with a=1, b=1
    return a + b


slow_sum(1, 1)  # NOTE: `slow_sum` will be called, obviously.

slow_sum.validate(2, 2)  # NOTE: Validate only.

try:
    slow_sum.validate(1, 'b')
except ValidationError as exc:
    print(exc)
    """
    1 validation error for SlowSum
    b
      value is not a valid integer (type=type_error.integer)
    """


Called with a=1, b=1
1 validation error for SlowSum
b
  value is not a valid integer (type=type_error.integer)


### Raw function

The raw function which was decorated is accessible, this is useful if in some scenarios you trust your input arguments and want to call the function in the most performant way.

In [11]:
from pydantic import validate_arguments


@validate_arguments
def repeat(s: str, count: int, *, separator: bytes = b'') -> bytes:
    b = s.encode()
    return separator.join(b for _ in range(count))


a = repeat('hello', 3)
print(a)
#> b'hellohellohello'

b = repeat.raw_function('good bye', 2, separator=b', ')  # NOTE: .raw_function() is what you want.
print(b)
#> b'good bye, good bye'


b'hellohellohello'
b'good bye, good bye'


* Async Functions: skipped.

### Custom Config

> The model behind `validate_arguments` can be customised using a config setting which is
> equivalent to setting the Config sub-class in normal models.

> ⚠️ The `fields` and `alias_generator` properties of `Config` which allow aliases to be configured are not 
> supported yet with `@validate_arguments`, using them will raise an error.

Configuration is set using the `config` keyword argument to the decorator, it may be either a config class or a dict of properties which are converted to a class later.

In [ ]:
from pydantic import ValidationError, validate_arguments


class Foobar:
    def __init__(self, v: str):
        self.v = v

    def __add__(self, other: 'Foobar') -> str:
        return f'{self} + {other}'

    def __str__(self) -> str:
        return f'Foobar({self.v})'


@validate_arguments(config=dict(arbitrary_types_allowed=True))
def add_foobars(a: Foobar, b: Foobar):
    return a + b


c = add_foobars(Foobar('a'), Foobar('b'))
print(c)
#> Foobar(a) + Foobar(b)

try:
    add_foobars(1, 2)
except ValidationError as e:
    print(e)
    """
    2 validation errors for AddFoobars
    a
      instance of Foobar expected (type=type_error.arbitrary_type;
    expected_arbitrary_type=Foobar)
    b
      instance of Foobar expected (type=type_error.arbitrary_type;
    expected_arbitrary_type=Foobar)
    """


### Limitations

**Important limitations**
* Remember that currently *pydantic* does coercion by default - bear that in mind!
* ⚠️ Function *return value* **NOT** validated (yet).

For a discussion of all limitations, read: https://docs.pydantic.dev/usage/validation_decorator/#limitations